# NFL Capstone:Modeling

### Starting Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
import sys

## Import datasets

In [3]:
year = pd.read_csv('../data/teamstarterdraft.csv')
yearAV = pd.read_csv('../data/teamstarterdraftAV.csv')
week = pd.read_csv('../data/weekstarterdraft.csv')
weekAV = pd.read_csv('../data/weekstarterdraftAV.csv')
yearnocoach = year.drop(columns=['coach', 'offcoor', 'defcoor', 'offscheme', 'defalign'])
yearnocoachAV = yearAV.drop(columns=['coach', 'offcoor', 'defcoor', 'offscheme', 'defalign'])

## New Datasets with bucketization of major categorical variables and ordinal encoding of weeks

In [4]:
weekord = week.copy()
weekAVord = weekAV.copy()
weekord['Week'] = weekord['Week'].replace('Wild Card', 18)
weekord['Week'] = weekord['Week'].replace('Division', 19)
weekord['Week'] = weekord['Week'].replace('Conf. Champ.', 20)
weekord['Week'] = weekord['Week'].replace('SuperBowl', 21)

weekAVord['Week'] = weekAVord['Week'].replace('Wild Card', 18)
weekAVord['Week'] = weekAVord['Week'].replace('Division', 19)
weekAVord['Week'] = weekAVord['Week'].replace('Conf. Champ.', 20)
weekAVord['Week'] = weekAVord['Week'].replace('SuperBowl', 21)

weekord['Week']= weekord['Week'].astype(int)
weekAVord['Week']= weekAVord['Week'].astype(int)

In [5]:
yearbucket = year.copy()
yearAVbucket = yearAV.copy()
top = yearbucket['coach'].isin(yearbucket['coach'].value_counts().index[:61])
yearbucket.loc[~top, 'coach'] = 'other'

top = yearbucket['offcoor'].isin(yearbucket['offcoor'].value_counts().index[:61])
yearbucket.loc[~top, 'offcoor'] = 'other'

top = yearbucket['defcoor'].isin(yearbucket['defcoor'].value_counts().index[:61])
yearAVbucket.loc[~top, 'defcoor'] = 'other'

top = yearAVbucket['coach'].isin(yearAVbucket['coach'].value_counts().index[:61])
yearAVbucket.loc[~top, 'coach'] = 'other'

top = yearAVbucket['offcoor'].isin(yearAVbucket['offcoor'].value_counts().index[:61])
yearAVbucket.loc[~top, 'offcoor'] = 'other'

top = yearAVbucket['defcoor'].isin(yearAVbucket['defcoor'].value_counts().index[:61])
yearAVbucket.loc[~top, 'defcoor'] = 'other'

# Model Selection Regression

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
#import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV


dfdict = {'year':year, 'yearAV':yearAV, 'week':week,'weekAV':weekAV, 'yearnocoach': yearnocoach, 'yearnocoachAV':yearnocoachAV}


dfresults= pd.DataFrame()
for key, df in dfdict.items():
        X = df.drop('DraftTeamSelection', axis=1)
        y = df['DraftTeamSelection']
        
        categorical_features = list(X.select_dtypes(include=['category', object]).columns)
        categorical_transformer = OneHotEncoder(sparse=True, handle_unknown='ignore')

        numeric_features = list(X.select_dtypes(include=['int', 'float']).columns)
        numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
        ])

        preprocessor = ColumnTransformer(
        transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)])
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        reg = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', LinearRegression())])
        search_space = [{'regressor': [LinearRegression()],
                        'regressor__normalize': [True, False]},
                        {'regressor':[Ridge()],
                        'regressor__alpha': np.logspace(-4, 0, 50),
                        'regressor__normalize': [True, False]},
                        {'regressor': [Lasso()],
                        'regressor__alpha': np.logspace(-4, 0, 50),
                        'regressor__normalize': [True, False]},
                        {'regressor': [ElasticNet()],
                        'regressor__l1_ratio': np.linspace(0,1,30),
                        'regressor__normalize': [True, False]},
                        {'regressor': [RandomForestRegressor()],
                        'regressor__n_estimators': np.logspace(2,3,20),
                        'regressor__max_depth': np.linspace(1,10,10),
                        'regressor__criterion': ['mse', 'mae']}]
        
        reg_CV = GridSearchCV(reg, search_space, cv=5, n_jobs=-1)
        best_model = reg_CV.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        
        df_best_regressor = best_model.best_estimator_.get_params()['regressor']
        R2 = r2_score(y_test, y_pred)
        MSE = mean_squared_error(y_test, y_pred)
        RMSE = mean_squared_error(y_test, y_pred, squared=False)
        MAE = mean_absolute_error(y_test, y_pred)
        results = pd.DataFrame({"R2": R2, 'MSE':MSE, 'RMSE': RMSE, 'MAE':MAE, 'best regressor': df_best_regressor}, index=[key])
        dfresults = dfresults.append(results)
print(dfresults)

                     R2       MSE      RMSE       MAE  \
year           0.003490  6.803684  2.608387  2.203176   
yearAV         0.000286  6.996596  2.645108  2.220828   
week           0.027038  6.751551  2.598375  2.174834   
weekAV         0.038667  6.563252  2.561884  2.143305   
yearnocoach    0.010850  6.734904  2.595169  2.183651   
yearnocoachAV  0.003437  6.907644  2.628240  2.184574   

                                                  best regressor  
year           ElasticNet(alpha=1.0, copy_X=True, fit_interce...  
yearAV         Lasso(alpha=0.15264179671752318, copy_X=True, ...  
week           Lasso(alpha=0.0009540954763499944, copy_X=True...  
weekAV         Lasso(alpha=0.0011513953993264468, copy_X=True...  
yearnocoach    Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...  
yearnocoachAV  Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...  


In [13]:
dfdict = {'year':year, 'yearAV':yearAV, 'week':week,'weekAV':weekAV}
dfresults= pd.DataFrame()
for key, df in dfdict.items():
        X = df.drop('DraftTeamSelection', axis=1)
        y = df['DraftTeamSelection']

        X_dummies = pd.get_dummies(X)
        print(key, X.shape, X_dummies.shape)


year (4261, 73) (4261, 497)
yearAV (3140, 82) (3140, 448)
week (71017, 34) (71017, 93)
weekAV (52331, 43) (52331, 102)


# Model Selection Regression w/ PCA

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import FunctionTransformer
#import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV


dfdict = {'year':year, 'yearAV':yearAV, 'week':week,'weekAV':weekAV, 'yearnocoach': yearnocoach, 'yearnocoachAV':yearnocoachAV}


dfresults= pd.DataFrame()
for key, df in dfdict.items():
        X = df.drop('DraftTeamSelection', axis=1)
        y = df['DraftTeamSelection']
        
        categorical_features = list(X.select_dtypes(include=['category', object]).columns)
        categorical_transformer = OneHotEncoder(sparse=False, handle_unknown='ignore')

        numeric_features = list(X.select_dtypes(include=['int', 'float']).columns)
        numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
        ])

        preprocessor = ColumnTransformer(
        transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)])
        
        pca = PCA()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        reg = Pipeline(steps=[('preprocessor', preprocessor),
                                ('pca', pca),
                                ('regressor', LinearRegression())])
        search_space = [{'pca__n_components':[2,15,30,50,100],
                        'regressor': [LinearRegression()],
                        'regressor__normalize': [True, False]},
                        {'pca__n_components':[2,15,30,50,100],
                        'regressor':[Ridge()],
                        'regressor__alpha': np.logspace(-4, 0, 50),
                        'regressor__normalize': [True, False]},
                        {'pca__n_components':[2,15,30,50,100],
                        'regressor': [Lasso()],
                        'regressor__alpha': np.logspace(-4, 0, 50),
                        'regressor__normalize': [True, False]},
                        {'pca__n_components':[2,15,30,50,100],
                        'regressor': [ElasticNet()],
                        'regressor__l1_ratio': np.linspace(0,1,30),
                        'regressor__normalize': [True, False]},
                        {'pca__n_components':[2,15,30,50,100],
                        'regressor': [RandomForestRegressor()],
                        'regressor__n_estimators': np.logspace(2,3,20),
                        'regressor__max_depth': np.linspace(1,10,10),
                        'regressor__criterion': ['mse', 'mae']}]
        
        reg_CV = GridSearchCV(reg, search_space, cv=5, n_jobs=-1)
        best_model = reg_CV.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        
        df_best_regressor = best_model.best_estimator_.get_params()['regressor']
        R2 = r2_score(y_test, y_pred)
        MSE = mean_squared_error(y_test, y_pred)
        RMSE = mean_squared_error(y_test, y_pred, squared=False)
        MAE = mean_absolute_error(y_test, y_pred)
        pca_components = reg_CV.best_estimator_.named_steps['pca'].n_components
        results = pd.DataFrame({'PCA components': pca_components, "R2": R2, 'MSE':MSE, 'RMSE': RMSE, 'MAE':MAE, 'best regressor': df_best_regressor}, index=[key])
        dfresults = dfresults.append(results)
print(dfresults)

               PCA components        R2       MSE      RMSE       MAE  \
year                       50  0.006157  6.564303  2.562090  2.148639   
yearAV                     50 -0.000032  7.133128  2.670792  2.246177   
week                       50  0.012887  6.832590  2.613922  2.188608   
weekAV                     50  0.016964  6.642123  2.577232  2.155551   
yearnocoach                50 -0.005583  7.053212  2.655788  2.221354   
yearnocoachAV               2 -0.004378  7.119864  2.668307  2.228804   

                                                  best regressor  
year           Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...  
yearAV         Lasso(alpha=0.0011513953993264468, copy_X=True...  
week           Ridge(alpha=0.007543120063354615, copy_X=True,...  
weekAV         Lasso(alpha=0.004291934260128779, copy_X=True,...  
yearnocoach    Ridge(alpha=0.6866488450042998, copy_X=True, f...  
yearnocoachAV  Ridge(alpha=0.15264179671752318, copy_X=True, ...  


# Model Selection Regresson run on datasets with bucketized coach values

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
#import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

#this is what I changed from cell above
dfdict = {'yearbucket':yearbucket, 'yearAVbucket':yearAVbucket, 'week':weekord,'weekAV':weekAVord}


dfresults= pd.DataFrame()
for key, df in dfdict.items():
        X = df.drop('DraftTeamSelection', axis=1)
        y = df['DraftTeamSelection']
        categorical_features = list(df.select_dtypes(include=['category', object]).columns)
        categorical_transformer = OneHotEncoder(sparse=True, handle_unknown='ignore')

        numeric_features = list(df.select_dtypes(include=['int', 'float']).columns)
        numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
        ])

        preprocessor = ColumnTransformer(
        transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)])
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        reg = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', LinearRegression())])
        search_space = [{'regressor': [LinearRegression()],
                        'regressor__normalize': [True, False]},
                        {'regressor':[Ridge()],
                        'regressor__alpha': np.logspace(-4, 0, 50),
                        'regressor__normalize': [True, False]},
                        {'regressor': [Lasso()],
                        'regressor__alpha': np.logspace(-4, 0, 50),
                        'regressor__normalize': [True, False]},
                        {'regressor': [ElasticNet()],
                        'regressor__l1_ratio': np.linspace(0,1,30),
                        'regressor__normalize': [True, False]},
                        {'regressor': [RandomForestRegressor()],
                        'regressor__n_estimators': np.logspace(2,3,20),
                        'regressor__max_depth': np.linspace(1,10,10),
                        'regressor__criterion': ['mse', 'mae']}]
        
        reg_CV = GridSearchCV(reg, search_space, cv=5, n_jobs=-1)
        best_model = reg_CV.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        
        df_best_regressor = best_model.best_estimator_.get_params()['regressor']
        R2 = r2_score(y_test, y_pred)
        MSE = mean_squared_error(y_test, y_pred)
        RMSE = mean_squared_error(y_test, y_pred, squared=False)
        MAE = mean_absolute_error(y_test, y_pred)
        results = pd.DataFrame({"R2": R2, 'MSE':MSE, 'RMSE': RMSE, 'MAE':MAE, 'best regressor':                 df_best_regressor}, index=[key])
        dfresults = dfresults.append(results)
print(dfresults)

                    R2       MSE      RMSE       MAE  \
yearbucket    0.002520  7.142888  2.672618  2.240276   
yearAVbucket  0.013071  6.455497  2.540767  2.151045   
week          0.024096  6.787724  2.605326  2.179035   
weekAV        0.041493  6.602885  2.569608  2.149873   

                                                 best regressor  
yearbucket    Lasso(alpha=0.0016768329368110067, copy_X=True...  
yearAVbucket  Lasso(alpha=0.0020235896477251557, copy_X=True...  
week          Ridge(alpha=0.04941713361323833, copy_X=True, ...  
weekAV        Ridge(alpha=0.040949150623804234, copy_X=True,...  


# Classification Model Selection

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import auc

dfdict = {'year':year, 'yearAV':yearAV, 'week':week,'weekAV':weekAV, 'yearnocoach': yearnocoach, 'yearnocoachAV':yearnocoachAV}


dfresults= pd.DataFrame()
for key, df in dfdict.items():
        X = df.drop('DraftPosition', axis=1)
        y = df['DraftPosition']
        #categorical_features = list(X.select_dtypes(include=['category', object]).columns)
        #categorical_transformer = make_pipeline(OneHotEncoder(sparse=True, handle_unknown='ignore'))

        numeric_features = list(X.select_dtypes(include=['int', 'float']).columns)
        numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
        ])

        preprocessor = ColumnTransformer(
        transformers=[
                ('num', numeric_transformer, numeric_features)])
                #('cat', categorical_transformer, categorical_features)])
        pca = PCA()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        clf = Pipeline(steps=[('preprocessor', preprocessor),
                                ('pca', pca),
                                ('classifier', RandomForestClassifier())])
        search_space = [{'pca__n_components':[2,15,30,50,100],
                        'classifier': [RandomForestClassifier()],
                        'classifier__n_estimators': [64,96,128,1000,2000],
                        'classifier__criterion': ['gini', 'entropy']},
                        {'pca__n_components':[2,15,30,50,100],
                        'classifier': [SVC()],
                        'classifier__C': [0.25, 0.50, 0.75, 1]}]
        
        clf_CV = GridSearchCV(clf, search_space, cv=5, n_jobs=-1)
        best_model = clf_CV.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
 
        ac = accuracy_score(y_test, y_pred)
        print(ac)
        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f1)
        #cm = confusion_matrix(y_test, y_pred)
        df_best_classifier = best_model.best_estimator_.get_params()['classifier']
        print(df_best_classifier)
        pca_components = clf_CV.best_estimator_.named_steps['pca'].n_components
        #,'best classifier': df_best_classifier
        results = pd.DataFrame({'PCA components': pca_components, "Accuracy": ac, "f1" : f1}, index=[key])
        dfresults = dfresults.append(results)
print(dfresults)

0.20540156361051884
0.08477457890001086
SVC(C=0.25, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.20154291224686596
0.06761230924493578
SVC(C=0.25, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.269115890083632
0.24101193239029695
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.2688477127967574
0.25192539054479235
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                    

# Best dataset (week) vs dummy classifier

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle


dfdict = {'week':week}


dfresults= pd.DataFrame()
for key, df in dfdict.items():
        X = df.drop('DraftPosition', axis=1)
        y = df['DraftPosition']
        #categorical_features = list(X.select_dtypes(include=['category', object]).columns)
        #categorical_transformer = make_pipeline(OneHotEncoder(sparse=True, handle_unknown='ignore'))

        numeric_features = list(X.select_dtypes(include=['int', 'float']).columns)
        scalers_to_test = [StandardScaler(), MinMaxScaler()]
        numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', MinMaxScaler())
        ])

        preprocessor = ColumnTransformer(
        transformers=[
                ('num', numeric_transformer, numeric_features)])
                #('cat', categorical_transformer, categorical_features)])
        pca = PCA()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        clf = Pipeline(steps=[('preprocessor', preprocessor),
                                ('pca', pca),
                                ('classifier', RandomForestClassifier())])
        search_space = [{'pca__n_components':[15,30],
                        'classifier': [RandomForestClassifier()],
                        'classifier__n_estimators': [64,96,128, 256],                         'classifier__max_depth': np.linspace(1,10,10),
                        'classifier__criterion': ['gini', 'entropy']},
                        {'pca__n_components':[15,30],
                        'classifier': [SVC()],
                        'classifier__C': [0.25, 1],
                        'classifier__kernel': ['poly', 'rbf']}]
        
        clf_CV = GridSearchCV(clf, search_space, cv=5, n_jobs=-1, verbose=10)
        best_model = clf_CV.fit(X_train, y_train)
        #saving model (https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)
        filename = 'best_model.sav'
        pickle.dump(best_model, open(filename, 'wb'))

Fitting 5 folds for each of 168 candidates, totalling 840 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   40.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  

In [15]:
from sklearn.dummy import DummyClassifier
y_pred = best_model.predict(X_test)

"""cv_results_df = pd.DataFrame(clf_CV.cv_results_)
cv_results_df = cv_results_df.sort_values(by=['rank_test_score'])
cv_results_df = (
            cv_results_df
            .set_index(cv_results_df["params"].apply(
            lambda x: "_".join(str(val) for val in x.values()))
            )
            .rename_axis('params')
    )
cv_results_df[
    ['params', 'rank_test_score', 'mean_test_score', 'std_test_score']
    ]"""
cv_results_df = pd.DataFrame(best_model.cv_results_)
cv_results_df = cv_results_df.sort_values(by=['rank_test_score'])
cv_results_df[
    ['params', 'rank_test_score', 'mean_test_score', 'std_test_score']
    ]
#Check how well model generalizes
y_train_pred = best_model.predict(X_train)
train_ac = accuracy_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

#dummy classifier
dummy_score_dict = {}
strategies = ['stratified', 'most_frequent', 'uniform']
for strategy in strategies:
        dclf = DummyClassifier(strategy=strategy)
        dclf.fit(X_train, y_train)
        dummy_score = dclf.score(X_test, y_test)
        dummy_score_dict[strategy] = dummy_score
dummy_df = pd.DataFrame.from_dict(dummy_score_dict, orient='index', columns=['score'])

ac = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
#cm = confusion_matrix(y_test, y_pred)
df_best_classifier = best_model.best_estimator_.get_params()['classifier']
pca_components = best_model.best_estimator_.named_steps['pca'].n_components
#,'best classifier': df_best_classifier
results = pd.DataFrame({'PCA components': pca_components,"Train Accuracy": train_ac, "Test Accuracy": ac, "Train F1":train_f1, "Test F1" : f1}, index=[key])
print("Cross-Validation best parameters: ", cv_results_df.iloc[0]['param_classifier'])
print("Cross-validation mean test score:", cv_results_df.iloc[0]['mean_test_score'])
dfresults = dfresults.append(results)
print(dummy_df)
print(dfresults)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
cv_results_df.head()

Cross-Validation best parameters:  SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Cross-validation mean test score: 0.29585346113896405
                  score
stratified     0.133555
most_frequent  0.201656
uniform        0.112306
      PCA components  Train Accuracy  Test Accuracy  Train F1   Test F1
week              30         0.37681       0.299667  0.352613  0.273753
[[2725  165  147  450  767   26  104   59  283]
 [ 784  413   34  198  465   15   72   40  199]
 [ 721   86  271  194  397   14   62   27  190]
 [ 921  106   94  826  635    4   94   54  228]
 [1166  148   76  397 1629   34  106   44  231]
 [ 389   45   45  179  257   53   27   34  111]
 [ 811  127   70  227  443   20  254   31  212]
 [ 558   77   44  134  236   12   71  140  102]
 [1121  116   96  331  515   23   72  

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__criterion,param_classifier__max_depth,param_classifier__n_estimators,param_pca__n_components,param_classifier__C,param_classifier__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
165,530.400788,3.799027,29.533797,0.111991,"SVC(C=1, break_ties=False, cache_size=200, cla...",NaN,NaN,NaN,30,1,poly,"{'classifier': SVC(C=1, break_ties=False, cach...",0.292319,0.297814,0.299075,0.298760,0.291299,0.295853,0.003344,1
167,509.903861,68.741612,32.315766,8.808143,"SVC(C=1, break_ties=False, cache_size=200, cla...",NaN,NaN,NaN,30,1,rbf,"{'classifier': SVC(C=1, break_ties=False, cach...",0.289797,0.294136,0.293611,0.297184,0.289723,0.292890,0.002832,2
154,56.801730,0.824380,0.265066,0.018032,"RandomForestClassifier(bootstrap=True, ccp_alp...",entropy,10,96,15,NaN,NaN,{'classifier': RandomForestClassifier(bootstra...,0.285699,0.278794,0.278373,0.288251,0.296238,0.285471,0.006613,3
156,77.130545,0.974941,0.358732,0.043102,"RandomForestClassifier(bootstrap=True, ccp_alp...",entropy,10,128,15,NaN,NaN,{'classifier': RandomForestClassifier(bootstra...,0.282442,0.279004,0.285204,0.294662,0.285729,0.285408,0.005207,4
152,41.386025,3.784504,0.219704,0.061618,"RandomForestClassifier(bootstrap=True, ccp_alp...",entropy,10,64,15,NaN,NaN,{'classifier': RandomForestClassifier(bootstra...,0.282232,0.275011,0.284363,0.290143,0.288672,0.284084,0.005359,5


In [20]:
dfresults = pd.DataFrame()

In [21]:
from sklearn.dummy import DummyClassifier
filename = 'check_best_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

y_pred = loaded_model.predict(X_test)

cv_results_df = pd.DataFrame(loaded_model.cv_results_)
cv_results_df = cv_results_df.sort_values(by=['rank_test_score'])
cv_results_df[
    ['params', 'rank_test_score', 'mean_test_score', 'std_test_score']
    ]
#Check how well model generalizes
y_train_pred = loaded_model.predict(X_train)
train_ac = accuracy_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

#dummy classifier
dummy_score_dict = {}
strategies = ['stratified', 'most_frequent', 'uniform']
for strategy in strategies:
        dclf = DummyClassifier(strategy=strategy)
        dclf.fit(X_train, y_train)
        dummy_score = dclf.score(X_test, y_test)
        dummy_score_dict[strategy] = dummy_score
dummy_df = pd.DataFrame.from_dict(dummy_score_dict, orient='index', columns=['score'])

ac = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
#cm = confusion_matrix(y_test, y_pred)
df_best_classifier = loaded_model.best_estimator_.get_params()['classifier']
pca_components = loaded_model.best_estimator_.named_steps['pca'].n_components
#,'best classifier': df_best_classifier
results = pd.DataFrame({'PCA components': pca_components,"Train Accuracy": train_ac, "Test Accuracy": ac, "Train F1":train_f1, "Test F1" : f1}, index=[key])
print("Cross-Validation best parameters: ", cv_results_df.iloc[0]['param_classifier'])
print("Cross-validation mean test score:", cv_results_df.iloc[0]['mean_test_score'])
dfresults = dfresults.append(results)
print(dummy_df)
print(dfresults)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
cv_results_df.head()

Cross-Validation best parameters:  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=96,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Cross-validation mean test score: 0.4415005660320774
                  score
stratified     0.131464
most_frequent  0.201656
uniform        0.109234
      PCA components  Train Accuracy  Test Accuracy  Train F1   Test F1
week              15        0.832265        0.82736  0.832153  0.827266
[[4073   62   67  101  145   27   80   34  137]
 [ 116 1800   21   45   98   24   36   18   62]
 [ 105   29 1594   50   65 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__criterion,param_classifier__n_estimators,param_pca__n_components,param_classifier__C,param_classifier__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,126.216762,3.148768,0.871261,0.147203,"RandomForestClassifier(bootstrap=True, ccp_alp...",entropy,96,15,NaN,NaN,{'classifier': RandomForestClassifier(bootstra...,0.403068,0.479088,0.455864,0.468894,0.400588,0.441501,0.033228,1
12,142.526362,3.419606,0.766433,0.037040,"RandomForestClassifier(bootstrap=True, ccp_alp...",entropy,128,15,NaN,NaN,{'classifier': RandomForestClassifier(bootstra...,0.401597,0.468264,0.452921,0.461118,0.414145,0.439609,0.026662,2
8,79.992735,7.473283,0.570158,0.064450,"RandomForestClassifier(bootstrap=True, ccp_alp...",entropy,64,15,NaN,NaN,{'classifier': RandomForestClassifier(bootstra...,0.403173,0.481505,0.440101,0.466898,0.401009,0.438537,0.032594,3
14,283.720571,6.574698,4.247592,1.514612,"RandomForestClassifier(bootstrap=True, ccp_alp...",entropy,256,15,NaN,NaN,{'classifier': RandomForestClassifier(bootstra...,0.395713,0.476776,0.442518,0.465847,0.396490,0.435469,0.033996,4
0,35.041457,0.947748,0.631997,0.116116,"RandomForestClassifier(bootstrap=True, ccp_alp...",gini,64,15,NaN,NaN,{'classifier': RandomForestClassifier(bootstra...,0.372701,0.467633,0.440942,0.437053,0.385351,0.420736,0.035870,5
